In [1]:
import pandas as pd

In [3]:
# Read gams file

time_cost = pd.read_csv(r'/Users/kmontocam/Downloads/Matriz_tiempos_TSP.csv', header = None) # csv file with the time cost of every route in minutes
output_path = r'/Users/kmontocam/Downloads/TSP.lst' # GAMS output

with open(output_path, encoding = 'utf-8') as f:
    output_gams = [line for line in f]

## Look for matrix variable

for current, further in zip(output_gams, output_gams[1:]):
    if 'binary variables' in current:
        var_name  = further
        var_name = var_name.partition('(')[0][::-1].partition(' ')[0]
        break

print(var_name)

x


In [4]:
# Look for results

for idx, line in enumerate(output_gams):
    if f'---- VAR {var_name}  \n' == line:
        start_from = idx + 4

routes_list_table = list()

for line in output_gams[start_from:]:
    if line == '\n':
        break
    if line.split()[-3] != '.':
        routes_list_table.append(line)

routes_list_table

['1 .12          .             1.0000         1.0000         2.9072      \n',
 '2 .10          .             1.0000         1.0000         6.8477      \n',
 '3 .25          .             1.0000         1.0000         5.9153      \n',
 '4 .2           .             1.0000         1.0000         5.5454      \n',
 '5 .18          .             1.0000         1.0000        12.7706      \n',
 '6 .16          .             1.0000         1.0000         3.0845      \n',
 '7 .1           .             1.0000         1.0000         1.8403      \n',
 '8 .5           .             1.0000         1.0000         0.4509      \n',
 '9 .4           .             1.0000         1.0000         4.9401      \n',
 '10.3           .             1.0000         1.0000         2.9928      \n',
 '11.14          .             1.0000         1.0000         0.2063      \n',
 '12.19          .             1.0000         1.0000        10.2287      \n',
 '13.11          .             1.0000         1.0000         0.9

In [67]:
# Keep indexes only and filter

routes_list = list()

for route in routes_list_table:
    if len(route.split()) == 6:
        route = route.split()[0] + route.split()[1]
        routes_list.append(route)
    else:
        routes_list.append(route.split()[0])

# Start list from home and order the route

routes_dict  = {route.split('.')[0]:route.split('.')[-1] for route in routes_list}

ordered_routes = [routes_list.pop()] # pops last item in route, since is the beginning of the track and begins sorted list

for _ in range(len(routes_list)):
    ordered_routes.append(
        str(routes_dict.get(ordered_routes[-1].split('.')[0])) + '.' +
        str(routes_dict.get(routes_dict.get(ordered_routes[-1].split('.')[0])))
        
        )

ordered_routes

['25.13',
 '13.11',
 '11.14',
 '14.7',
 '7.1',
 '1.12',
 '12.19',
 '19.20',
 '20.23',
 '23.6',
 '6.16',
 '16.21',
 '21.17',
 '17.15',
 '15.22',
 '22.24',
 '24.8',
 '8.5',
 '5.18',
 '18.9',
 '9.4',
 '4.2',
 '2.10',
 '10.3',
 '3.25']

In [52]:
# Calculate time

MAX_LABOR_HOURS = 8 # Hours the driver can work during the day
STOP_AT_UNIV = 30 # Minutes that the driver will stay in the university

labor_time = int()
total_time = int()
day = 1
journey = list()

for route in ordered_routes:

    row, col = route.split('.')

    visit_time = time_cost.iloc[int(row) - 1, int(col) -1] + STOP_AT_UNIV # Add the time that it takes to get from one point to another and add 30 minutes for the sale

    if labor_time + visit_time <= 60 * MAX_LABOR_HOURS:
        
        labor_time += visit_time
        journey.append([day, route, visit_time, labor_time])
    else:

        day += 1
        journey.append([day, route, visit_time, labor_time])
        labor_time = 0
    
    total_time += visit_time

In [60]:
route = pd.DataFrame(journey, columns = ['day', 'visit', 'time_spent_in_visit', 'minutes_worked_in_day'])
route.to_csv('route.csv', index = False)

In [45]:
total_time/60 # La persona logra realizar el ruteo en dos jornadas laborales exactas

15.983333333333333

In [63]:
print(route.to_latex(index = False, multirow = True))

\begin{tabular}{rlrr}
\toprule
 day & visit &  time\_spent\_in\_visit &  minutes\_worked\_in\_day \\
\midrule
   1 & 25.13 &                   35 &                     35 \\
   1 & 13.11 &                   32 &                     67 \\
   1 & 11.14 &                   30 &                     97 \\
   1 &  14.7 &                   34 &                    131 \\
   1 &   7.1 &                   34 &                    165 \\
   1 &  1.12 &                   35 &                    200 \\
   1 & 12.19 &                   48 &                    248 \\
   1 & 19.20 &                   55 &                    303 \\
   1 & 20.23 &                   69 &                    372 \\
   1 &  23.6 &                   34 &                    406 \\
   1 &  6.16 &                   36 &                    442 \\
   1 & 16.21 &                   36 &                    478 \\
   2 & 21.17 &                   32 &                    478 \\
   2 & 17.15 &                   34 &                     

In [65]:
total_time/60

15.983333333333333